In [1]:
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings(action='ignore')

일반적인 선형 회귀 분석은 예측 문제를 풀기에는 적합하지만 분류 문제를 풀기에는 적합하지 않다.

선형 회귀 분석의 타겟 데이터의 값의 범위에는 제한이 없기 때문에 결과값이 제한되는 상황에서 회귀 모형이 결과값에 제한이 없다먼 분류 문제를 풀기 어려워진다. 0과 1로 분류해야 하는 문제에서 모형 결과값이 오직 0과 1 사이의 값으로 나와야 할 것이다. 이러한 문제점을 해결하기 위해 사용하는 방법 로지스틱 회귀 분석이다.

$$z = w^Tx + b$$

위 식은 선형 회귀 분석 모형이다.  
$z$ 값에는 제한이 없으므로 어떤 값도 가질수 있으므로 위 식을 이용해서 분류 문제를 푸는 것은 어렵다는 것을 알 수 있다. 기존의 선형 회귀 모형 식을 분류 문제를 풀 수 있도록 변형시키는 과정이 필요하다. 이를 위해 결과값이 제한된 범위의 값을 가지도록 변형시켜 보자.

$$y = \frac{1}{1 + e^{-z}} = \frac{1}{1 + e^{-(w^Tx + b)}}$$

$z$에 대한 선형 회귀 식을 위와 같이 변형시키면 새로운 출렷 $y$는 0과 1 사이의 값만 가지게 된다. 위 함수를 시그모이드(sigmoid) 함수라고 부른다. 시그모이드 함수는 딥러닝에서도 자주 나오는 함수로 위 식의 우변이 $z = w^Tx + b$ 형태가 되도록 변형시키면 아래와 같이 표현할 수 있다.

$$log(\frac{y}{1 - y}) = w^Tx + b$$

위 식을 보면 우변이 $w^Tx + b$와 같이 선형 형태로 나타난다. 위 식에서 좌변의 $\frac{y}{1 - y}$를 오즈 비(odds ratio)라고 부른다.  
분자에 해당하는 $y$가 사건이 발생할 확률(혹은 성공 확률)이라고 했을 때 분모인 $1 - y$ 값은 사건이 발생하지 않을 확률(혹은 실패 확률)에 해당된다. 실패 확률과 성공 확률의 비를 오즈 비라고 부른다. 또한 오즈 비에 log를 취한 값을 로짓(logit)이라고 부른다.

로지스틱 회귀 실습

In [2]:
# 데이터 불러오기
from sklearn import datasets # 위스콘신 암 데이터를 사용하기 위해 import 한다.
raw_cancer = datasets.load_breast_cancer() # 위스콘신 암 데이터를 불러온다.

In [3]:
# 피쳐/타겟 데이터 지정
X = raw_cancer.data # 위스콘신 암 피쳐 데이터를 저장한다.
y = raw_cancer.target # 위스콘신 암 타겟 데이터를 저장한다.

In [4]:
# 트레이닝/테스트 데이터 분할
from sklearn.model_selection import train_test_split # 트레이닝/테스트 데이터 분할을 위해 import 한다.
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=1) # 트레이닝 데이터와 테스트 데이터로 분할한다.
print(X_tn.shape, X_te.shape)

(426, 30) (143, 30)


In [5]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler # 데이터 표준화를 위해 import 한다.
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
# 표준화는 트레이닝 데이터를 기반으로 실행하므로 트레이닝 피쳐 데이터 X_tn을 표준화 스케일러에 적합시킨다.
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn) # 트레이닝 피쳐 데이터 X_tn을 표준화 한다.
X_te_std = std_scale.transform(X_te) # 테스트 피쳐 데이터 X_te를 표준화 한다.

In [6]:
# 데이터 학습
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀 분석 알고리즘을 사용하기 위해 import 한다.
# 로지스틱 회귀 분석 모델 객체를 만든다.
# 선형 회귀 분석과 마찬가지로 제약식을 적용할 수 있다. penalty 옵션을 사용해 제약식 종류를 지정한다.
# L1 제약식을 적용하고 싶다면 penalty='l1', L2 제약식을 적용하고 싶다면 penalty='l2', 엘라스틱 넷을 적용하고 
# 싶다면 penalty='elasticnet', 제약식을 적용하고 싶지 않다면 penalty='none'이라고 입력한다.
clf_logi = LogisticRegression(penalty='l2')
# 표준화된 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn을 넣어서 선형 회귀 분석 알고리즘을 학습시킨다.
clf_logi.fit(X_tn_std, y_tn)

LogisticRegression()

In [7]:
# 로지스틱 회귀 분석 계수, 상수항 확인
print(clf_logi.coef_) # 가중치
print(clf_logi.intercept_) # 바이어스

[[-0.48143978 -0.41014209 -0.46725987 -0.51392775 -0.09769918  0.3020924
  -0.72884664 -0.7890253  -0.01424461  0.39299676 -1.08415325 -0.04728618
  -0.5669909  -0.74499028 -0.22229903  0.90896096 -0.05386185 -0.49962018
   0.17826258  0.71184481 -1.05809159 -0.95888641 -0.91097127 -0.97646535
  -0.41365059 -0.01729729 -0.82465772 -0.93295021 -0.76852329 -0.61983664]]
[0.36341572]


In [8]:
# 데이터 예측 - 클래스로 예측
pred_logistic = clf_logi.predict(X_te_std) # 표준화된 테스트 데이터 X_te_std로 예측한다.
print(pred_logistic)

[1 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0 0 1 1 0
 1 0 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0
 1 0 0 1 1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 0 1 0 0 1 1 1 0 1 0 0 1 1 0 0 0 1]


In [9]:
# 데이터 예측 - 클래스에 속할 확률로 예측
# predict_proba() 함수를 이용해 해당 클래스에 속할 확률을 예측한다.
# 암 데이터는 두 가지(양성, 악성) 클래스로 예측되므로 두 개의 열로 결과가 이루어진 것을 확인할 수 있다.
pred_proba = clf_logi.predict_proba(X_te_std)
print(pred_proba)

[[1.49488533e-01 8.50511467e-01]
 [9.89656817e-01 1.03431830e-02]
 [9.87187552e-03 9.90128124e-01]
 [9.94890005e-01 5.10999525e-03]
 [8.94370796e-01 1.05629204e-01]
 [9.94653457e-01 5.34654285e-03]
 [9.99934190e-01 6.58097367e-05]
 [8.39850104e-01 1.60149896e-01]
 [6.71620121e-03 9.93283799e-01]
 [2.48219387e-02 9.75178061e-01]
 [1.16133334e-03 9.98838667e-01]
 [9.97966542e-01 2.03345776e-03]
 [9.81618379e-01 1.83816207e-02]
 [1.87756245e-03 9.98122438e-01]
 [3.41727616e-01 6.58272384e-01]
 [3.22621426e-02 9.67737857e-01]
 [5.94546271e-04 9.99405454e-01]
 [8.11730977e-03 9.91882690e-01]
 [6.99424444e-04 9.99300576e-01]
 [9.99931832e-01 6.81684178e-05]
 [2.52926669e-03 9.97470733e-01]
 [4.58182448e-03 9.95418176e-01]
 [9.99946728e-01 5.32715029e-05]
 [1.28963362e-02 9.87103664e-01]
 [9.92283615e-01 7.71638528e-03]
 [3.06703381e-02 9.69329662e-01]
 [2.40952072e-02 9.75904793e-01]
 [9.99991486e-01 8.51390903e-06]
 [1.00000000e+00 2.62520146e-13]
 [9.85741614e-01 1.42583858e-02]
 [9.999999

In [10]:
# 정밀도 평가
from sklearn.metrics import precision_score # 정밀도를 측정하기 위해 import 한다.
# precision_score() 함수의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_logistic)를 넘겨 정밀도를 평가한다.
percision = precision_score(y_te, pred_logistic)
print(percision)

0.9775280898876404


In [11]:
# 혼돈 행렬 확인
from sklearn.metrics import confusion_matrix # 혼돈 행렬을 만들기 위해 import 한다.
# confusion_matrix() 함수에 실제 타겟 데이터와 예측된 데이터를 넘겨 혼돈 행렬을 만든다.
conf_matrix = confusion_matrix(y_te, pred_logistic)
print(conf_matrix)

[[53  2]
 [ 1 87]]


In [12]:
# 분류 리포트 확인
from sklearn.metrics import classification_report # 분류 리포트를 출력하기 위해 import 한다.
# classification_report() 함수에 실제 타겟 데이터와 예측된 데이터를 넘겨 분류 리포트를 만든다.
class_report = classification_report(y_te, pred_logistic)
print(class_report)

              precision    recall  f1-score   support

           0       0.98      0.96      0.97        55
           1       0.98      0.99      0.98        88

    accuracy                           0.98       143
   macro avg       0.98      0.98      0.98       143
weighted avg       0.98      0.98      0.98       143

